In [1]:
import math
import pickle
import torch
import torch.nn as nn
import numpy as np
import math, random
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import matplotlib
import matplotlib.pyplot as plt

In [2]:
batch_size = 100

mnist_train = datasets.MNIST('./MNIST/', train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test  = datasets.MNIST('/MNIST/', train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=mnist_test,batch_size=1,shuffle=True)

In [19]:
#Encoder
class Encoder_net(nn.Module):
    def __init__(self):
        super(Encoder_net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(X_dim, N),
            nn.Dropout(p=prob),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(N, N),
            nn.Dropout(p=prob),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(N, N),
            nn.Dropout(p=prob),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(N, N),
            nn.Dropout(p=prob),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(N, N0),
            nn.Dropout(p=prob),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(N0, z_dim)        
            )
    def forward(self, x):
        xgauss = self.layers(x)
        return xgauss

In [20]:
# Decoder
class Decoder_net(nn.Module):
    def __init__(self):
        super(Decoder_net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(z_dim, N0),
            nn.Dropout(p=prob),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(N0, N),
            nn.Dropout(p=prob),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(N, N),
            nn.Dropout(p=prob),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(N, N),
            nn.Dropout(p=prob),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(N, N),
            nn.Dropout(p=prob),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(N, X_dim),
            nn.Sigmoid()
            )
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [21]:
def Corrupt_Gaussian(Image, sigma):
    print(Image.size())
    GN = Image + torch.normal(torch.zeros(Image.size()))*sigma
    GN[GN<0] = 0
    GN[GN>1] = 1
    return GN

In [1]:
prob = 0.2
X_dim = 784
N = 600
N0 = 150
z_dim = 2

torch.manual_seed(10)
Encoder = Encoder_net()
Decoder = Decoder_net()     # Encoder/Decoder

Encoder = Encoder.cuda()
Decoder = Decoder.cuda()
# Set learning rates
gen_lr, reg_lr = 0.0004, 0.0008
# Set optimizators
Encoder_grad = optim.Adam(Encoder.parameters(), lr=gen_lr)
Decoder_grad = optim.Adam(Decoder.parameters(), lr=gen_lr)

criterionL2 = nn.MSELoss().cuda()

TINY = 1e-8
LossEpoch = []

for i in range(1):
    if i ==0:
        print('Start')
    loss = []
    for batch, label in train_loader:
        X = batch.view([100,1,784])
        X_corrupt = Corrupt_Gaussian(X,3)
        X = Variable(X).cuda()
        X_corrupt = Variable(X_corrupt).cuda()
        z_sample = Encoder(X_corrupt)
        X_sample = Decoder(z_sample)
        recon_loss = criterionL2(X_sample, X)
        Encoder_grad.zero_grad()
        Decoder_grad.zero_grad()
        recon_loss.backward()
        Encoder_grad.step()
        Decoder_grad.step()
        loss.append(recon_loss.data)
    l = np.mean(np.array(loss)).cpu().numpy()
    print(i ,l[0])
    LossEpoch.append(l)

NameError: name 'torch' is not defined